## Branch and Bound Method for Integer Programming

First we import these modules

In [20]:
import scipy
import numpy as np
import itertools

#### Branch and Bound

In [21]:
def branch_and_bound(c, A, b, lb, ub, isMax = False, depth = 1):
    # c: profit vector
    # isMax: whether it is a maximization or minimization problem
    # lb, ub: vectors indicate the lower bound and upper bound of all decision variables
    # depth: the depth of the tree

    n = len(c)

    # Optimal solution for LP relaxation
    res = scipy.optimize.linprog(-c if isMax else c, A, b, bounds = list(zip(lb, ub)))

    # Check if LP is feasible
    if not res.success:
        return [], -np.inf if isMax else np.inf, depth
    
    # Candidate for the optimal value and the objective value
    x_candidate, z_candidate = res.x, res.fun
    z_candidate = -z_candidate if isMax else z_candidate
    # Correct sign for maximization

    is_int = True 
    for i in range(n):
        # If x_i is not integer
        if not np.isclose(x_candidate[i],round(x_candidate[i])):
            is_int = False
            break

    if is_int:
        return x_candidate, z_candidate, depth
    
    # Branch on the first non-integer variable

    for i in range(n):
        if not np.isclose(x_candidate[i], round(x_candidate[i])):
            # Left branch: x_i <= floor(x_i)
            left_lb = np.copy(lb)
            left_ub = np.copy(ub)
            left_ub[i] = np.floor(x_candidate[i])
            x_left, z_left, depth_left = branch_and_bound(c, A, b, left_lb, left_ub, isMax, depth + 1)

            # Right branch: x_i >= ceil(x_i)
            right_lb = np.copy(lb)     
            right_ub = np.copy(ub) 
            right_lb[i] = np.ceil(x_candidate[i])
            x_right, z_right, depth_right = branch_and_bound(c, A, b, right_lb, right_ub, isMax, depth + 1)

            # Return the best solution
            # For max problem, return the branch with greater z
            if isMax:
                if(z_left > z_right):
                    return x_left, z_left, depth_left
                else: 
                    return x_right, z_right, depth_right
            # For min problem, return the branch with smaller z
            else:
                if(z_left < z_right):
                    return x_left, z_left, depth_left
                else: 
                    return x_right, z_right, depth_right            

#### Constraint vector A

In [22]:
def matrix_A(num_vars):
    # Number of columns is the number of variables
    cols = num_vars
    # Number of rows is the number of all possible pair of the variables
    rows = round(num_vars*(num_vars-1)/2)
    A = np.zeros((rows, cols))

    # All possible permutations of column entry
    permutations = list(itertools.combinations(range(cols), 2))

    # Generate matrix A by letting the two column entries in each permutation (row) equal to 1 
    for i in range(len(permutations)):
        A[i][permutations[i][0]] = 1
        A[i][permutations[i][1]] = 1
    return A

print(matrix_A(4))

[[1. 1. 0. 0.]
 [1. 0. 1. 0.]
 [1. 0. 0. 1.]
 [0. 1. 1. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 1.]]


#### Test

In [23]:
# Objective Function Coefficient 
c = np.array([20,20,100,70,60])

# Constraint
A = matrix_A(5)
b = np.ones(10)


# Lower bound for each constraint 
lb = [0, 0, 0, 0, 0]
# Upper bound for each constraint 
ub = [1, 1, 1, 1, 1]

x_optimal, f_optimal, depth_optimal = branch_and_bound(c, A, b, lb, ub, True)
print("========= Optimal Solutions =========")

print("Variable:")
for idx, var in enumerate(x_optimal): 
    print(f">  x{idx} = {var}")

print("Objective Function:")
print(f">  f = {f_optimal}")
print(f"Tree depth: {depth_optimal}")

========= Optimal Solutions =========
Variable:
>  x0 = 0.0
>  x1 = 0.0
>  x2 = 1.0
>  x3 = -0.0
>  x4 = -0.0
Objective Function:
>  f = 100.0
Tree depth: 4
